In [1]:
import datasets

In [2]:
ds = datasets.load_dataset("JeffreyXiang/TRELLIS-500K", 'ObjaverseXL (github)')

README.md:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

ObjaverseXL_github.csv:   0%|          | 0.00/318M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/311843 [00:00<?, ? examples/s]

In [8]:
import os
# os.environ['ATTN_BACKEND'] = 'xformers'   # Can be 'flash-attn' or 'xformers', default is 'flash-attn'
os.environ['SPCONV_ALGO'] = 'native'        # Can be 'native' or 'auto', default is 'auto'.
                                            # 'auto' is faster but will do benchmarking at the beginning.
                                            # Recommended to set to 'native' if run only once.

from PIL import Image
from TRELLIS.trellis.pipelines import TrellisImageTo3DPipeline

In [13]:
# model_id = "JeffreyXiang/TRELLIS-image-large"
model_id = "Stable-X/trellis-normal-v0-1"
pipeline = TrellisImageTo3DPipeline.from_pretrained(model_id)
pipeline.cuda()

pipeline.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

ss_dec_conv3d_16l8_fp16.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

ss_dec_conv3d_16l8_fp16.safetensors:   0%|          | 0.00/148M [00:00<?, ?B/s]

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-68158276-452912b20845e050366854da;445df8f2-ad89-4cbf-b338-825ff0855dd3)

Repository Not Found for url: https://huggingface.co/ckpts/ss_dec_conv3d_16l8_fp16/resolve/main/.json.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication

In [11]:
image = Image.open("sample.jpg")

outputs = pipeline.run(
    image,
    seed=1,
    # Optional parameters
    # sparse_structure_sampler_params={
    #     "steps": 12,
    #     "cfg_strength": 7.5,
    # },
    # slat_sampler_params={
    #     "steps": 12,
    #     "cfg_strength": 3,
    # },
)
# outputs is a dictionary containing generated 3D assets in different formats:
# - outputs['gaussian']: a list of 3D Gaussians
# - outputs['radiance_field']: a list of radiance fields
# - outputs['mesh']: a list of meshes

NameError: name 'pipeline' is not defined

In [ ]:
filename = "sample.glb"

# TODO: find another way ; postprocessing requires NVIDIA kaolin which is not compatible with my version of torch + CUDA
glb = postprocessing_utils.to_glb(
    outputs['gaussian'][0],
    outputs['mesh'][0],
    # Optional parameters
    simplify=0.95,          # Ratio of triangles to remove in the simplification process
    texture_size=1024,      # Size of the texture used for the GLB
)
glb.export(filename)

In [ ]:
import k3d
import vtk
import ipywidgets as widgets

reader = vtk.vtkGLTFReader() 
reader.SetFileName(filename)
reader.Update() 

plot = k3d.plot()
mb = reader.GetOutput()

iterator = mb.NewIterator()

vtk_polyobjects = []
while not iterator.IsDoneWithTraversal():
    item = iterator.GetCurrentDataObject()
    vtk_polyobjects.append(item)
    iterator.GoToNextItem()

    
for obj in vtk_polyobjects:
    plot += k3d.vtk_poly_data(obj, color=0x222222)
plot.display()

debug_info = widgets.HTML()

In [1]:
%load_ext gradio